In [1]:
import sys
import os

project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
import torch
import torch.nn as nn

from src.utils.data import (
    load_data_100k, load_data_1m, load_data_monti
)

from src.GLocalKernel.kernel import KernelNet
from src.GLocalKernel.model import GlobalLocalKernelModel
from src.GLocalKernel.train import GLKTrainer

In [3]:
# Set the project root directory
PROJECT_ROOT = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), "."))

# Set the paths to the datasets
MOVIELENS_100K_PATH = os.path.join(PROJECT_ROOT, "data", "MovieLens_100K")
MOVIELENS_1M_PATH = os.path.join(PROJECT_ROOT, "data", "MovieLens_1M")
DOUBAN_MONTI_PATH = os.path.join(PROJECT_ROOT, "data", "DoubanMonti")

# Load the MovieLens 100k dataset
n_m, n_u, train_r, train_m, test_r, test_m = load_data_100k(
    path=MOVIELENS_100K_PATH,
)

.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~
Number of users: 943
Number of movies: 1682
Number of training ratings: 80000
Number of test ratings: 20000
.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~


In [4]:
kernel_net = KernelNet(
    n_u = n_u,
    n_layers = 2,
    n_hid = 500,
    n_dim = 5,
    lambda_s = 6e-3,
    lambda_2 = 20.,
    activation = nn.Sigmoid(),
    dropout_rate = 0.33,
)
    
model = GlobalLocalKernelModel(
    local_kernel_net = kernel_net,
    gk_size = 3,
    dot_scale = 1.,
    n_m = n_m,
)

In [ ]:
trainer = GLKTrainer(
    model = model,
    optimizer_p = torch.optim.AdamW(model.local_kernel_net.parameters(), lr=1e-3),
    optimizer_f = torch.optim.AdamW(model.parameters(), lr=1e-3),
    epoch_p = 500,
    epoch_f = 1000,
)

In [ ]:
train_r_local = trainer.pre_train_model(
    train_r = torch.Tensor(train_r),
    train_m = torch.Tensor(train_m),
    test_r = torch.Tensor(test_r),
    test_m = torch.Tensor(test_m),
    tol_p = 1e-4,
    patience_p = 5
)

Pre-training epoch:  0 Train RMSE:  2.7324414 Test RMSE:  2.738648
~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.
Pre-training epoch:  5 Train RMSE:  1.9859825 Test RMSE:  2.0237033
~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.
Pre-training epoch:  10 Train RMSE:  1.2453213 Test RMSE:  1.2885828
~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.
Pre-training epoch:  15 Train RMSE:  1.3074372 Test RMSE:  1.3253527
~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.
Pre-training epoch:  20 Train RMSE:  1.1134534 Test RMSE:  1.1486149
~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.
Pre-training epoch:  25 Train RMSE:  1.1031222 Test RMSE:  1.1433395
~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.

In [7]:
trainer.fine_tune_model(
    train_r = torch.Tensor(train_r),
    train_r_local = torch.Tensor(train_r_local.clip(0, 5)),
    train_m = torch.Tensor(train_m),
    test_r = torch.Tensor(test_r),
    test_m = torch.Tensor(test_m),
    tol_f = 1e-5,
    patience_f = 10
)

Fine-tuning epoch:  0 
 Train RMSE:  1.0387824 Train MAE:  0.8307246 Train NDCG:  0.8243536991775259 
 Test RMSE:  1.0808713 Test MAE:  0.8637088 Test NDCG:  0.8238709398661929
~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.
Fine-tuning epoch:  20 
 Train RMSE:  0.94437647 Train MAE:  0.75162894 Train NDCG:  0.8831971803376545 
 Test RMSE:  0.9889381 Test MAE:  0.786978 Test NDCG:  0.8796773120360954
~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.
Fine-tuning epoch:  40 
 Train RMSE:  0.8974445 Train MAE:  0.7110993 Train NDCG:  0.8954554735836343 
 Test RMSE:  0.95166576 Test MAE:  0.75330955 Test NDCG:  0.8867693622395364
~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.~.
Fine-tuning epoch:  60 
 Train RMSE:  0.87849426 Train MAE:  0.6949262 Train NDCG:  0.8991719726464958 
 Test RMSE:  0.93695223 Test MAE:  0.7395976 Test NDCG:  